In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as sum_, first, round, min, when, lit, to_date, to_timestamp, date_format, hour
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, TimestampType

In [2]:
!pip install boto3 pandas

import boto3
import pandas as pd
import io

s3_output_path = 's3a://bucket-raw-upa-connect-eduardo/tabela_historico_sensor.csv'

csv_files = [
    'bucket-raw-upa-connect-eduardo/dados_2025_05_20.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_21.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_22.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_23.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_24.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_25.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_26.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_27.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_28.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_29.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_30.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_05_31.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_06_01.csv',
    'bucket-raw-upa-connect-eduardo/dados_2025_06_02.csv'
]


# Inicializa o cliente S3
s3_client = boto3.client('s3')

# Lista para armazenar os DataFrames
dfs = []

for file in csv_files:
    bucket_name, key = file.split('/', 1)
    
    # Faz o download do arquivo
    response = s3_client.get_object(Bucket=bucket_name, Key=key)
    
    # Lê o CSV diretamente do objeto S3 (response['Body'] é um StreamingBody)
    df = pd.read_csv(io.BytesIO(response['Body'].read()))
    
    dfs.append(df)

# Concatena todos os DataFrames em um só
df_final = pd.concat(dfs, ignore_index=True)
df_final['data_hora'] = pd.to_datetime(df_final['data_hora'])

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [3]:
# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

TabelaCompleta = spark.createDataFrame(df_final)

df_camera = TabelaCompleta.filter(
    (col("fk_sensor") == 1)
).filter(col("valor").isNotNull())


df_camera_filtrado = df_camera.withColumn("data_hora", to_timestamp(col("data_hora"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("qtde_pessoas", col("valor").cast(IntegerType())) \
    .withColumn("fk_upa", col("fk_upa").cast(IntegerType())) \
    .drop("valor") \
    .drop("fk_unid_medida") \
    .drop("fk_paciente") \
    .drop("fk_sensor")

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a9710dcd-7584-416a-992b-570880f7e1f9;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 506ms :: artifacts dl 22ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

In [4]:
df_camera_filtrado = df_camera_filtrado.filter(
        (col("qtde_pessoas") >= 0) & (col("qtde_pessoas") <= 200)
    ) \
    .orderBy(col("fk_upa"))

In [5]:
df_final_semanal = df_camera_filtrado.withColumn("data", to_date(col("data_hora"))) \
                .drop("data_hora")  

df_final_semanal = df_final_semanal.groupBy('data', 'fk_upa').agg(
    avg('qtde_pessoas').cast(IntegerType()).alias('media_pessoas')
).orderBy('fk_upa', 'data')


df_final_diario = df_camera_filtrado.withColumn("data", to_date(col("data_hora"))) \
                .withColumn("hora", hour(col("data_hora"))) \
                .drop("data_hora")                

df_final_diario = df_final_diario.groupBy('data', 'hora', 'fk_upa').agg(
    avg('qtde_pessoas').cast(IntegerType()).alias('media_pessoas')
).orderBy('fk_upa', 'data', 'hora')


In [9]:
import boto3

S3_BUCKET_CREDENTIALS = 'bucket-trusted-upa-connect-eduardo'
S3_KEY_CREDENTIALS = 'credenciais.json'
LOCAL_CREDENTIALS_PATH = '/tmp/credenciais.json'

print(f"Baixando credenciais do S3: s3://{S3_BUCKET_CREDENTIALS}/{S3_KEY_CREDENTIALS} para {LOCAL_CREDENTIALS_PATH}...")
s3 = boto3.client('s3')
try:
    s3.download_file(S3_BUCKET_CREDENTIALS, S3_KEY_CREDENTIALS, LOCAL_CREDENTIALS_PATH)
    print("Credenciais baixadas com sucesso.")
except Exception as e:
    print(f"Erro ao baixar credenciais do S3: {e}")
    sys.exit(1)

Baixando credenciais do S3: s3://bucket-trusted-upa-connect-eduardo/credenciais.json para /tmp/credenciais.json...
Credenciais baixadas com sucesso.


/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [10]:
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"'{package}' instalado com sucesso.")
    except Exception as e:
        print(f"Erro ao instalar '{package}': {e}")

# Instalar as bibliotecas necessárias
install_package("pandas")
install_package("google-auth-oauthlib")

!pip uninstall gspread -y
!pip install gspread==3.7.0

import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os
import sys

GOOGLE_SHEET_ID = '1i6BfuZXPOcTp6BFAiVkpktOBym0HtakZacUKfDzu1zI'


# === AUTENTICAÇÃO GOOGLE ===


scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credenciais = Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH, scopes=scopes)
cliente = gspread.authorize(credenciais)
planilha = cliente.open_by_key(GOOGLE_SHEET_ID)

# === ENVIA O DATAFRAME FINAL PARA A NOVA ABA "teste" ===
NOME_ABA_VISAO_COMPUTACIONAL_SEMANAL = "VisaoComputacionalSemanal"
NOME_ABA_VISAO_COMPUTACIONAL_DIARIO = "VisaoComputacionalDiario"

# Converter Spark DataFrame para Pandas DataFrame
try:
    df_pandas_semanal = df_final_semanal.toPandas()
    df_pandas_diario = df_final_diario.toPandas()
    print("DataFrame Spark convertido para Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao converter Spark DataFrame para Pandas: {e}")
    print("O DataFrame pode ser muito grande para a memória do driver ou a sessão Spark está inválida.")
    sys.exit(1)

# --- NOVO TRECHO: Ajustar tipagem das colunas para o Google Sheets ---
# Apenas formata colunas de data/hora para string ISO 8601.
# As demais colunas (numéricas, strings, etc.) são mantidas em seus tipos nativos no Pandas.
# O gspread e o Google Sheets farão a inferência de tipo para elas.

# Coluna 'data_chegada': assegurar que é string DD/MM/YYYY
if 'data' in df_pandas_semanal.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas_semanal['data']):
        df_pandas_semanal['data'] = df_pandas_semanal['data'].dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    elif pd.api.types.is_string_dtype(df_pandas_semanal['data']):
        # Tentar converter strings que podem não estar em formato padrão para datetime,
        # e depois formatar para DD/MM/YYYY. 'errors=coerce' transforma não-datas em NaT.
        df_pandas_semanal['data'] = pd.to_datetime(df_pandas_semanal['data'], errors='coerce').dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    # Preencher valores nulos (NaT) com string vazia, pois None também pode causar problemas.
    df_pandas_semanal['data'] = df_pandas_semanal['data'].fillna('')
    
# Coluna 'data_chegada': assegurar que é string DD/MM/YYYY
if 'data' in df_pandas_diario.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas_diario['data']):
        df_pandas_diario['data'] = df_pandas_diario['data'].dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    elif pd.api.types.is_string_dtype(df_pandas_diario['data']):
        # Tentar converter strings que podem não estar em formato padrão para datetime,
        # e depois formatar para DD/MM/YYYY. 'errors=coerce' transforma não-datas em NaT.
        df_pandas_diario['data'] = pd.to_datetime(df_pandas_diario['data'], errors='coerce').dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    # Preencher valores nulos (NaT) com string vazia, pois None também pode causar problemas.
    df_pandas_diario['data'] = df_pandas_diario['data'].fillna('')

    
dados_semanais = [df_pandas_semanal.columns.tolist()] + df_pandas_semanal.values.tolist()
dados_diarios = [df_pandas_diario.columns.tolist()] + df_pandas_diario.values.tolist()

try:
    aba = planilha.worksheet(NOME_ABA_VISAO_COMPUTACIONAL_SEMANAL)
    aba.clear()
    print(f"Aba '{NOME_ABA_VISAO_COMPUTACIONAL_SEMANAL}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba = planilha.add_worksheet(title=NOME_ABA_VISAO_COMPUTACIONAL_SEMANAL, rows=str(len(dados_semanais) + 100), cols=str(len(df_pandas_semanal.columns) + 10))
    print(f"Aba '{NOME_ABA_VISAO_COMPUTACIONAL_SEMANAL}' criada.")

try:
    aba.update(dados_semanais)
    print(f"Dados do DataFrame final enviados com sucesso para a aba '{NOME_ABA_VISAO_COMPUTACIONAL_SEMANAL}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets: {e}")

    
try:
    aba = planilha.worksheet(NOME_ABA_VISAO_COMPUTACIONAL_DIARIO)
    aba.clear()
    print(f"Aba '{NOME_ABA_VISAO_COMPUTACIONAL_DIARIO}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba = planilha.add_worksheet(title=NOME_ABA_VISAO_COMPUTACIONAL_DIARIO, rows=str(len(dados_diarios) + 100), cols=str(len(df_pandas_semanal.columns) + 10))
    print(f"Aba '{NOME_ABA_VISAO_COMPUTACIONAL_DIARIO}' criada.")

try:
    aba.update(dados_diarios)
    print(f"Dados do DataFrame final enviados com sucesso para a aba '{NOME_ABA_VISAO_COMPUTACIONAL_DIARIO}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets: {e}")

# Opcional: Remover o arquivo de credenciais temporário
if os.path.exists(LOCAL_CREDENTIALS_PATH):
    os.remove(LOCAL_CREDENTIALS_PATH)
    print(f"Arquivo de credenciais temporário '{LOCAL_CREDENTIALS_PATH}' removido.")


'pandas' instalado com sucesso.


'google-auth-oauthlib' instalado com sucesso.
Found existing installation: gspread 3.7.0
Uninstalling gspread-3.7.0:
  Successfully uninstalled gspread-3.7.0
  Using cached gspread-3.7.0-py3-none-any.whl.metadata (1.9 kB)
Using cached gspread-3.7.0-py3-none-any.whl (28 kB)
DataFrame Spark convertido para Pandas com sucesso.
Aba 'VisaoComputacionalSemanal' encontrada e limpa.
Dados do DataFrame final enviados com sucesso para a aba 'VisaoComputacionalSemanal'.
Aba 'VisaoComputacionalDiario' encontrada e limpa.
Dados do DataFrame final enviados com sucesso para a aba 'VisaoComputacionalDiario'.
Arquivo de credenciais temporário '/tmp/credenciais.json' removido.
